In [ ]:
import os
os.chdir('..')
os.chdir('..')
import scvelo as scv
import scanpy as sc
import cell2fate as c2f
import pickle as pickle
from eval_utils import cross_boundary_correctness
from datetime import datetime
import pandas as pd
import numpy as np
from os.path import exists
import matplotlib.pyplot as plt
import torch
import unitvelo as utv
import scvi
method = 'Cell2fateDynamicalModel_1000epochs_10samples_seed999'
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'
datasets = ['Pancreas_with_cc',  'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']
n_genes_list = np.array((2000, 3000))
n_counts_list = np.array((10, 20))

for i in range(len(n_genes_list)):
    for j in range(len(n_counts_list)):
        for k in (3,1,2,0,4):
            print(i)
            print(j)
            print(k)
            dataset = datasets[k]
            n_genes = n_genes_list[i]
            min_counts = n_counts_list[j]
            model_index = str(i) + '-' + str(j) + '-' + str(k)
            save_name = method + '_'
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
                if model_index in tab.index:
                    continue
            adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
            adata = c2f.utils.get_training_data(adata, cells_per_cluster = 10**6, cluster_column = 'clusters',
                                            remove_clusters = [], min_shared_counts = min_counts, n_var_genes= n_genes)
            scvi.settings.seed = 999
            c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label='spliced', unspliced_label='unspliced')    
            n_modules = c2f.utils.get_max_modules(adata)
            mod = c2f.Cell2fate_DynamicalModel(adata,
                                               n_modules = n_modules)   
            mod.train(max_epochs = 1000)
            sample_kwarg = {"num_samples": 10, "batch_size" : adata.n_obs,
                 "use_gpu" : True, 'return_samples': False}
            adata = mod.export_posterior(adata, sample_kwargs = sample_kwarg)
            mod.compute_and_plot_total_velocity_scvelo(adata, save = False, delete = False)
            # Calculate performance metrics:
            file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
            ground_truth = pickle.load(file)
            metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
            cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                        for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
            if exists(save_dir + save_name + '_CBDC_fullBenchmark.csv'):
                tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
            else:
                c_names = ['CBDC', 'ELBO5', 'ELBO10', 'ELBO20', 'ELBO40']
                tab = pd.DataFrame(columns = c_names)
            tab.loc[model_index, 'CBDC'] = np.mean(cb_score)
            tab.loc[model_index, 'ELBO40'] = np.mean(mod.history['elbo_train'][-40:])[0]
            tab.loc[model_index, 'ELBO20'] = np.mean(mod.history['elbo_train'][-20:])[0]
            tab.loc[model_index, 'ELBO10'] = np.mean(mod.history['elbo_train'][-10:])[0]
            tab.loc[model_index, 'ELBO5'] = np.mean(mod.history['elbo_train'][-5:])[0]
            tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')
tab = pd.read_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv', index_col = 0)
tab.loc['AVERAGE', 'CBDC'] = np.mean(tab['CBDC'])
tab.to_csv(save_dir + save_name + '_CBDC_fullBenchmark.csv')

Global seed set to 0


(Running UniTVelo 0.2.5)
2023-10-30 13:38:18
0
0
3
Keeping at most 1000000 cells per cluster
Filtered out 46616 genes that are detected 10 counts (shared).


Global seed set to 999


Extracted 2000 highly variable genes.
Leiden clustering ...
Number of Leiden Clusters: 9
Maximal Number of Modules: 10


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

# Cross-Boundary Direction Correctness (A->B)
{('nIPC', 'Neuroblast'): -0.11137499289720569, ('Neuroblast', 'Granule immature'): 0.5351878158527363, ('Granule immature', 'Granule mature'): 0.31636692598941824, ('Radial Glia-like', 'Astrocytes'): 0.7252722135670729, ('OPC', 'OL'): 0.5253559578521401}
Total Mean: 0.3981615840728324
# In-cluster Coherence
{'Astrocytes': 0.9451231, 'Cajal Retzius': 0.7008574, 'Cck-Tox': 0.87399566, 'Endothelial': 0.90484303, 'GABA': 0.84085685, 'Granule immature': 0.7929305, 'Granule mature': 0.86159533, 'Microglia': 0.9774817, 'Mossy': 0.60527027, 'Neuroblast': 0.75956446, 'OL': 0.983324, 'OPC': 0.86569667, 'Radial Glia-like': 0.7974065, 'nIPC': 0.50424504}
Total Mean: 0.8152278661727905
1
0
2
Keeping at most 1000000 cells per cluster


Global seed set to 999


Filtered out 19601 genes that are detected 10 counts (shared).
Skip filtering by dispersion since number of variables are less than `n_top_genes`.
Leiden clustering ...


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Number of Leiden Clusters: 8
Maximal Number of Modules: 9
Epoch 487/1000:  49%|██████████████████████████████████████████████████████████████████████████▎                                                                              | 486/1000 [03:44<03:54,  2.19it/s, v_num=1, elbo_train=3.7e+6]